In [24]:
# Data Augmentation in Pytorch

In [25]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
import torchvision.datasets as datasets
from torch.utils.data import DataLoader as dataloader
import torchvision.models as models

import time
import os
import random
import numpy as np
import matplotlib.pyplot as plt
from tqdm.notebook import trange, tqdm
import copy

from Trainer import ModelTrainer

In [26]:
# Hyperparameters
batch_size = 64
num_epochs = 32
learning_rate = 1e-4
dataset_root = "../datasets"
image_size = 96

In [27]:
start_from_checkpoint = False
save_dir = "../data/Models"
model_name = "ResNet18_STL10"

In [28]:
# Set device to GPU_indx if PU is available
GPU_indx = 0
device = torch.device(GPU_indx if torch.cuda.is_available() else "cpu")

In [29]:
# Prepare a composition of transforms
# All models from the Pytorch model Zoo where trained using images normalised with 
# the mean and std (one per channel) of the whole ImageNet Dataset
# therefore the pretrained feature "detectors" of the model will expect the input to 
# be normalized in the same way 
# https://pytorch.org/docs/stable/torchvision/models.html
train_transform = transforms.Compose([
    transforms.Resize(image_size),
    #transforms.AutoAugment(),
    transforms.ToTensor(),
    #transforms.RandomRotation(degrees=10),
    #transforms.RandomHorizontalFlip(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

In [ ]:
# Define our STL10 Datasets
# https://pytorch.org/docs/stable/torchvision/datasets.html#torchvision.datasets.STL10

# Dataset definition is a bit differenet to MNIST and CIFAR10
# STL10 has 3 different datasets, test, train and unlabeled
# http://ai.stanford.edu/~acoates/stl10/
# training set only has 5000 images and test set only 8000
# Image size in this dataset are 96x96, larger then what we've been using

train_data = datasets.STL10(dataset_root, split='train', download=True, transform=train_transform)
test_data = datasets.STL10(dataset_root, split="test", download=True)

# Split trainging data into train and validation set with 90/10% traning/validation split
validation_split = 0.9

n_train_examples = int(len(train_data) * validation_split)
n_valid_examples = len(train_data) - n_train_examples
n_train_examples = int(len(train_data)*validation_split)
n_valid_examples = len(train_data) - n_train_examples

train_data, valid_data = torch.utils.data.random_split(train_data, [n_train_examples, n_valid_examples], generator=torch.Generator().manual_seed(42))


 30%|███       | 797M/2.64G [11:56<38:58, 788kB/s]     

In [ ]:
# Create an instance of a classifier model
model = models.resnet18()

In [ ]:

# Load a pre-trained model!
# SEE EXAMPLE 2 FOR MORE INFO!

# pretrained_model_name = "ResNet18_STL10_puzzle"
# pretrained_model_trainer = ModelTrainer(model=model, output_size=24, device="cpu", 
#                                         loss_fun=nn.CrossEntropyLoss(), 
#                                         batch_size=batch_size, learning_rate=learning_rate, 
#                                         save_dir=save_dir, model_name=pretrained_model_name, 
#                                         start_from_checkpoint=True)

# model = copy.deepcopy(pretrained_model_trainer.model)

In [ ]:

model_trainer = ModelTrainer(model=model.to(device), output_size=10, device=device, 
                             loss_fun=nn.CrossEntropyLoss(), 
                             batch_size=batch_size, learning_rate=learning_rate, 
                             save_dir=save_dir, model_name=model_name, 
                             start_from_checkpoint=start_from_checkpoint)

In [ ]:
model_trainer.set_data(train_set=train_data, test_set=test_data, val_set=valid_data)

In [ ]:
# Uncomment one of these and try it out!

model_trainer.set_lr_schedule(optim.lr_scheduler.StepLR(model_trainer.optimizer, 
                                                         step_size=1, 
                                                         gamma=0.95))

# model_trainer.set_lr_schedule(optim.lr_scheduler.CosineAnnealingLR(model_trainer.optimizer, 
#                                                                    T_max=num_epochs, 
#                                                                    eta_min=0))

In [ ]:
plt.figure(figsize = (20,10))
images, labels = next(iter(model_trainer.train_loader))
out = torchvision.utils.make_grid(images[0:16], normalize=True)
_ = plt.imshow(out.numpy().transpose((1, 2, 0)))

In [ ]:
# Lets see how many Parameter's our Model has!
num_params = 0
for param in model_trainer.model.parameters():
    num_params += param.flatten().shape[0]
print("This model has %d (approximately %d Million) Parameters!" % (num_params, num_params//1e6))

In [ ]:
model_trainer.run_training(num_epochs=num_epochs)

In [ ]:
print("The highest validation accuracy was %.2f%%" %(model_trainer.best_valid_acc*100))

In [ ]:

_ = plt.figure(figsize = (10,5))
train_x = np.linspace(0, num_epochs, len(model_trainer.train_loss_logger))
_ = plt.plot(train_x, model_trainer.train_loss_logger)
_ = plt.title("Training Loss")

In [ ]:

_ = plt.figure(figsize = (10,5))
train_x = np.linspace(0, num_epochs, len(model_trainer.train_acc_logger))
_ = plt.plot(train_x, model_trainer.train_acc_logger, c = "y")
valid_x = np.linspace(0, num_epochs, len(model_trainer.val_acc_logger))
_ = plt.plot(valid_x, model_trainer.val_acc_logger, c = "k")

_ = plt.title("Accuracy")
_ = plt.legend(["Training accuracy", "Validation accuracy"])

In [ ]:

# Call the evaluate function and pass the evaluation/test dataloader etc
test_acc = model_trainer.evaluate_model(train_test_val="test")
print("The Test Accuracy is: %.2f%%" %(test_acc*100))